In [3]:
import numpy as np
import pandas as pd
import requests
import json
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

## Import elastic V0 data

In [9]:
df_test = pd.read_csv("./data/results_matomo_v0.csv", dtype=str)

In [10]:
df_test = df_test.drop(columns=["url", "Status", "Commentaire"], axis=1)

In [11]:
df_test.head(3)

,terms,siren,results_elastic,pages_elastic,siren_elastic,resp_time_elastic,results_postgres,pages_postgres,siren_postgres,resp_time_postgres,results_postges,pages_postges,rank_elastic,rank_postgres
0,stellantis,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.063221,1,1,['879786085'],0.042676,1,1,0,0
1,STELLANTIS,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.146982,1,1,['879786085'],0.065058,1,1,0,0
2,Stellantis,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.084343,1,1,['879786085'],0.042565,1,1,0,0


In [12]:
df_test_2 = pd.read_csv("./data/results_nginx_v0.csv", dtype=str)

In [13]:
df_test_2 = df_test_2.drop(
    columns=[
        "url_post",
        "url_elastic",
        "Google",
        "Pappers",
        "siret",
        "degree of condifence",
    ],
    axis=1,
)

In [14]:
df_test_2.head(3)

,terms,siren,results_elastic,pages_elastic,siren_elastic,resp_time_elastic,results_postgres,pages_postgres,siren_postgres,resp_time_postgres,results_postges,pages_postges,rank_elastic,rank_postgres
0,club mediterranee,572185684,90,5,"['379834906', '782615843', '482003191', '84191...",0.217592,84,5,"['782615843', '394049233', '514659853', '48200...",0.095049,84,5,-1,-1
1,emmaus communaute,304233505,111,6,"['538548157', '304233505', '311686497', '34015...",0.13071,45,3,"['342642071', '399498690', '340155571', '78590...",0.051059,45,3,1,12
2,emmaus communaute,782901599,111,6,"['538548157', '304233505', '311686497', '34015...",0.141893,45,3,"['342642071', '399498690', '340155571', '78590...",0.063611,45,3,14,17


In [15]:
df_test_2.columns

Index(['terms', 'siren', 'results_elastic', 'pages_elastic', 'siren_elastic',
       'resp_time_elastic', 'results_postgres', 'pages_postgres',
       'siren_postgres', 'resp_time_postgres', 'results_postges',
       'pages_postges', 'rank_elastic', 'rank_postgres'],
      dtype='object')

In [16]:
df_test.columns

Index(['terms', 'siren', 'results_elastic', 'pages_elastic', 'siren_elastic',
       'resp_time_elastic', 'results_postgres', 'pages_postgres',
       'siren_postgres', 'resp_time_postgres', 'results_postges',
       'pages_postges', 'rank_elastic', 'rank_postgres'],
      dtype='object')

In [17]:
df_test.shape

(739, 14)

In [18]:
df_test_2.shape

(189, 14)

In [19]:
df_test = pd.concat([df_test, df_test_2], ignore_index=True)

In [20]:
df_test.shape

(928, 14)

In [21]:
df_test.dtypes

terms                 object
siren                 object
results_elastic       object
pages_elastic         object
siren_elastic         object
resp_time_elastic     object
results_postgres      object
pages_postgres        object
siren_postgres        object
resp_time_postgres    object
results_postges       object
pages_postges         object
rank_elastic          object
rank_postgres         object
dtype: object

In [22]:
df_test.head(3)

,terms,siren,results_elastic,pages_elastic,siren_elastic,resp_time_elastic,results_postgres,pages_postgres,siren_postgres,resp_time_postgres,results_postges,pages_postges,rank_elastic,rank_postgres
0,stellantis,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.063221,1,1,['879786085'],0.042676,1,1,0,0
1,STELLANTIS,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.146982,1,1,['879786085'],0.065058,1,1,0,0
2,Stellantis,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.084343,1,1,['879786085'],0.042565,1,1,0,0


In [23]:
df_test.shape

(928, 14)

## Add results V1 (avec enseignes et adresses enseignes)

In [24]:
def find(key, dictionary):
    for k, v in dictionary.items():
        if k == key:
            yield v
        elif isinstance(v, dict):
            for result in find(key, v):
                yield result
        elif isinstance(v, list):
            for d in v:
                for result in find(key, d):
                    yield result

In [25]:
def get_response(url, q):
    params["q"] = q
    response = requests.get(url, params=params)
    time_elapsed = response.elapsed.total_seconds()
    content = json.loads(response.content)
    total_results = content[0]["total_results"]
    total_pages = content[0]["total_pages"]
    siren_list = list(find("siren", content[0]))
    return total_results, total_pages, siren_list, time_elapsed

In [26]:
url_elastic = "http://api.sirene.dataeng.etalab.studio/search"

In [27]:
params = {"q": "", "page": "1", "per_page": "20"}

In [28]:
(
    df_test["results_elastic_1"],
    df_test["pages_elastic_1"],
    df_test["siren_elastic_1"],
    df_test["resp_time_elastic_1"],
) = ("", "", "", "")

In [29]:
df_test

,terms,siren,results_elastic,pages_elastic,siren_elastic,resp_time_elastic,results_postgres,pages_postgres,siren_postgres,resp_time_postgres,results_postges,pages_postges,rank_elastic,rank_postgres,results_elastic_1,pages_elastic_1,siren_elastic_1,resp_time_elastic_1
0,stellantis,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.063221,1,1,['879786085'],0.042676,1,1,0,0,,,,
1,STELLANTIS,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.146982,1,1,['879786085'],0.065058,1,1,0,0,,,,
2,Stellantis,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.084343,1,1,['879786085'],0.042565,1,1,0,0,,,,
3,air france,420495178,706,36,"['420495178', '314119504', '378006027', '77568...",0.271584,788,40,"['883454803', '824718910', '401867015', '84159...",0.255068,788,40,0,6,,,,
4,Air france,420495178,706,36,"['420495178', '314119504', '378006027', '77568...",0.067208,788,40,"['883454803', '824718910', '401867015', '84159...",0.104733,788,40,0,6,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,pontecaille gregoire,818452476,1,1,['818452476'],0.141171,1,1,['818452476'],0.039299,1,1,0,0,,,,
924,sherpa,393825229,2870,144,"['393825229', '801698242', '500913785', '33407...",0.103189,330,17,"['344857321', '484513593', '398939744', '33813...",0.07168,330,17,0,15,,,,
925,sherpa,500913785,2870,144,"['393825229', '801698242', '500913785', '33407...",0.137308,330,17,"['344857321', '484513593', '398939744', '33813...",0.070307,330,17,2,-1,,,,
926,mc animation,820552925,3,1,"['820552925', '440736965', '398015974']",0.047684,4,1,"['820552925', '440736965', '398015974', '50865...",0.047149,4,1,0,0,,,,


In [30]:
for index, row in df_test.iterrows():
    (
        df_test["results_elastic_1"][index],
        df_test["pages_elastic_1"][index],
        df_test["siren_elastic_1"][index],
        df_test["resp_time_elastic_1"][index],
    ) = get_response(url_elastic, row["terms"])

In [33]:
df_test["results_elastic"] = df_test["results_elastic"].astype("int32")
df_test["pages_elastic"] = df_test["pages_elastic"].astype("int32")
df_test["resp_time_elastic"] = df_test["resp_time_elastic"].astype("float64")
df_test["results_postgres"] = df_test["results_postgres"].astype("int32")
df_test["pages_postgres"] = df_test["pages_postgres"].astype("int32")
df_test["resp_time_postgres"] = df_test["resp_time_postgres"].astype("float64")

In [31]:
df_test["results_elastic_1"] = df_test["results_elastic_1"].astype("int32")
df_test["pages_elastic_1"] = df_test["pages_elastic_1"].astype("int32")
df_test["resp_time_elastic_1"] = df_test["resp_time_elastic_1"].astype("float64")

In [34]:
df_test.describe()

,results_elastic,pages_elastic,resp_time_elastic,results_postgres,pages_postgres,resp_time_postgres,results_elastic_1,pages_elastic_1,resp_time_elastic_1
count,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000,928.000000
mean,1192.389009,60.338362,0.115209,97.755388,5.580819,0.076688,1224.755388,61.913793,0.062363
std,2865.904545,143.351552,0.073632,372.932707,18.511431,0.066871,2878.074095,143.965180,0.031763
min,0.000000,1.000000,0.033094,0.000000,0.000000,0.005441,0.000000,1.000000,0.018051
25%,2.000000,1.000000,0.067629,1.000000,1.000000,0.038499,3.000000,1.000000,0.040811
50%,12.000000,1.000000,0.091565,3.000000,1.000000,0.050410,23.000000,2.000000,0.053917
75%,234.000000,12.000000,0.130058,15.000000,1.000000,0.079544,266.250000,14.000000,0.077933
max,10000.000000,501.000000,0.524740,2000.000000,100.000000,0.350441,10000.000000,501.000000,0.324647


## Ranks

In [45]:
df_test["rank_elastic_1"] = ""

In [46]:
for ind, row in df_test.iterrows():
    if str(row["siren"]) in row["siren_elastic_1"]:
        df_test["rank_elastic_1"][ind] = row["siren_elastic_1"].index(str(row["siren"]))
    else:
        df_test["rank_elastic_1"][ind] = -1

In [37]:
# df_test['rank_elastic_1'] = df_test['rank_elastic_1'].astype('int32')

## KPIs

In [47]:
fig = px.histogram(
    df_test.sort_values(by=["rank_elastic_1"]),
    x="rank_elastic_1",
    color_discrete_sequence=["indianred"],
    title="Distribution elastic des rangs du bon résultat",
)
fig.update_layout(bargap=0.5)
fig.update_xaxes(type="category")
fig.show()

In [48]:
x_elastic = df_test.sort_values(by=["rank_elastic"])["rank_elastic"]
x_postgres = df_test.sort_values(by=["rank_postgres"])["rank_postgres"]
x_elastic_1 = df_test.sort_values(by=["rank_elastic_1"])["rank_elastic_1"]
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        histfunc="count",
        x=x_elastic_1,
        name="Elasticsearch_1",
        marker_color="#eb0e3e",
    )
)
fig.add_trace(
    go.Histogram(
        histfunc="count", x=x_elastic, marker_color="#EB89B5", name="Elasticsearch"
    )
)
fig.add_trace(
    go.Histogram(
        histfunc="count",
        x=x_postgres,
        name="Postgres",
        marker_color="#330C73",
    )
)

fig.update_layout(
    title_text="Fréquence des rangs des résultats de la recherche",  # title of plot
    xaxis_title_text="Rang du résulat dans la page",  # xaxis label
    yaxis_title_text="Nombre de requêtes",  # yaxis label
    bargap=0.2,  # gap between bars of adjacent location coordinates
    bargroupgap=0.1,  # gap between bars of the same location coordinates
)
fig.update_xaxes(type="category")
fig.show()

In [49]:
fig = px.pie(
    df_test,
    names="rank_elastic_1",
    hole=0.7,
    color="rank_elastic_1",
    title="Pourcentages des rangs du bon résultat dans la recherche elasticsearch",
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

In [50]:
fig = px.pie(
    df_test,
    names="rank_elastic",
    hole=0.7,
    color="rank_elastic",
    title="Pourcentages des rangs du bon résultat dans la recherche elasticsearch",
)
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

In [51]:
df_test.to_csv("./data/elastic_wars.csv", header=True, index=False)

In [53]:
df_test[df_test["rank_elastic_1"] == -1]["rank_elastic"].value_counts()

-1    68
0      4
8      2
1      1
Name: rank_elastic, dtype: int64

In [58]:
df_test[(df_test["rank_elastic_1"] == -1) & (df_test["rank_elastic"] == "0")]

,terms,siren,results_elastic,pages_elastic,siren_elastic,resp_time_elastic,results_postgres,pages_postgres,siren_postgres,resp_time_postgres,results_postges,pages_postges,rank_elastic,rank_postgres,results_elastic_1,pages_elastic_1,siren_elastic_1,resp_time_elastic_1,rank_elastic_1
405,CENTRE HOSPITALIER SAINTE MARIE NICE,792718215,1,1,['792718215'],0.065832,4,1,"['792718215', '662025196', '775672272', '54209...",0.095065,4,1,0,0,102,6,"[379587900, 345086177, 380129866, 418096392, 8...",0.096142,-1
406,Centre hospitalier sainte Marie nice,792718215,1,1,['792718215'],0.074315,4,1,"['792718215', '662025196', '775672272', '54209...",0.242998,4,1,0,0,102,6,"[379587900, 345086177, 380129866, 418096392, 8...",0.101886,-1
407,CENTRE HOSPITALIER SAINT MARIE NICE,792718215,1,1,['792718215'],0.149439,4,1,"['792718215', '662025196', '775672272', '54209...",0.117831,4,1,0,0,102,6,"[379587900, 345086177, 380129866, 418096392, 8...",0.086617,-1
914,via pierre 1,314438631,52,3,"['314438631', '344207766', '851893503', '51121...",0.070852,18,1,"['314438631', '344207766', '511217408', '83000...",0.108642,18,1,0,0,155,8,"[356000000, 519037584, 552081317, 662042449, 3...",0.131888,-1


In [105]:
df_test[(df_test["rank_elastic_1"] == -1) & (df_test["rank_elastic"] == 0)]

,terms,siren,results_elastic,pages_elastic,siren_elastic,resp_time_elastic,results_postgres,pages_postgres,siren_postgres,resp_time_postgres,results_postges,pages_postges,rank_elastic,rank_postgres,results_elastic_1,pages_elastic_1,siren_elastic_1,resp_time_elastic_1,rank_elastic_1


In [4]:
df_test = pd.read_csv("./data/elastic_wars.csv", dtype=str)

In [5]:
df_test

,terms,siren,results_elastic,pages_elastic,siren_elastic,resp_time_elastic,results_postgres,pages_postgres,siren_postgres,resp_time_postgres,results_postges,pages_postges,rank_elastic,rank_postgres,results_elastic_1,pages_elastic_1,siren_elastic_1,resp_time_elastic_1,rank_elastic_1
0,stellantis,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.063221,1,1,['879786085'],0.042676,1,1,0,0,1353,68,"['879786085', '803902485', '379118748', '75231...",0.164481,0.0
1,STELLANTIS,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.146982,1,1,['879786085'],0.065058,1,1,0,0,1353,68,"['879786085', '803902485', '379118748', '75231...",0.105351,0.0
2,Stellantis,879786085,1353,68,"['879786085', '803902485', '879147148', '34219...",0.084343,1,1,['879786085'],0.042565,1,1,0,0,1353,68,"['879786085', '803902485', '379118748', '75231...",0.066599,0.0
3,air france,420495178,706,36,"['420495178', '314119504', '378006027', '77568...",0.271584,788,40,"['883454803', '824718910', '401867015', '84159...",0.255068,788,40,0,6,1129,57,"['420495178', '632041042', '314119504', '37800...",0.100744,0.0
4,Air france,420495178,706,36,"['420495178', '314119504', '378006027', '77568...",0.067208,788,40,"['883454803', '824718910', '401867015', '84159...",0.104733,788,40,0,6,1129,57,"['420495178', '632041042', '314119504', '37800...",0.128663,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,pontecaille gregoire,818452476,1,1,['818452476'],0.141171,1,1,['818452476'],0.039299,1,1,0,0,1,1,['818452476'],0.060331,0.0
924,sherpa,393825229,2870,144,"['393825229', '801698242', '500913785', '33407...",0.103189,330,17,"['344857321', '484513593', '398939744', '33813...",0.07168,330,17,0,15,2982,150,"['801698242', '515198844', '449614189', '39382...",0.106538,3.0
925,sherpa,500913785,2870,144,"['393825229', '801698242', '500913785', '33407...",0.137308,330,17,"['344857321', '484513593', '398939744', '33813...",0.070307,330,17,2,-1,2982,150,"['801698242', '515198844', '449614189', '39382...",0.070771,4.0
926,mc animation,820552925,3,1,"['820552925', '440736965', '398015974']",0.047684,4,1,"['820552925', '440736965', '398015974', '50865...",0.047149,4,1,0,0,5,1,"['820552925', '440736965', '398015974', '50865...",0.044872,0.0
